### Capstone Project - The Battle of Neighborhoods

## Business Problem section

# Background
According to Bloomberg News, the London Housing Market is in a rut. It is now facing a number of different headwinds, including the prospect of higher taxes and a warning from the Bank of England that U.K. home values could fall as much as 30 percent in the event of a disorderly exit from the European Union. More specifically, four overlooked cracks suggest that the London market may be in worse shape than many realize: hidden price falls, record-low sales, homebuilder exodus and tax hikes addressing overseas buyers of homes in England and Wales.

# Business Problem
In this scenario, it is urgent to adopt machine learning tools in order to assist homebuyers clientele in London to make wise and effective decisions. As a result, the business problem we are currently posing is: how could we provide support to homebuyers clientele in to purchase a suitable real estate in London in this uncertain economic and financial scenario?

To solve this business problem, we are going to cluster London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We will recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

# Data section
Data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). The following fields comprise the address data included in Price Paid Data: Postcode; PAON Primary Addressable Object Name. Typically the house number or name; SAON Secondary Addressable Object Name. If there is a sub-building, for example, the building is divided into flats, there will be a SAON; Street; Locality; Town/City; District; County.

To explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we will access data through FourSquare API interface and arrange them as a dataframe for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we will be able to recommend profitable real estate investments.

 # Methodology section
The Methodology section will describe the main components of our analysis and predication system. The Methodology section comprises four stages:

1. Collect Inspection Data
2. Explore and Understand Data
3. Data preparation and preprocessing 
4. Modeling

# 1. Collect Inspection Data
After importing the necessary libraries, we download the data from the HM Land Registry website as follows

In [1]:
import os # Operating System
import numpy as np
import pandas as pd
import datetime as dt # Datetime
import json # library to handle JSON files

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

!conda install -c conda-forge folium=0.5.0 --yes
import folium #import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs:
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    geopy-1.18.1               |             py_0          51 KB  conda-forge
    ------------------------------------------------------------
                                           Total:          84 KB

The following NEW packages will be INSTALLED:

  geographiclib      conda-forge/noarch::geographiclib-1.49-py_0

The following packages will be UPDATED:

  geopy              conda-forge/linux-64::geopy-1.11.0-py~ --> conda-forge/noarch::geopy-1.18.1-py_0



geopy-1.18.1         | 51 KB     | ##################################### | 100% 
geographiclib-1.49   | 32 KB     | ##################################### | 100% 
Preparing tran

In [2]:
#Read the data for examination (Source: http://landregistry.data.gov.uk/)
df_ppd = pd.read_csv("http://prod2.publicdata.landregistry.gov.uk.s3-website-eu-west-1.amazonaws.com/pp-2018.csv")

we anayize the data before using it

In [3]:
df_ppd.head(5)

,{68FEB20B-A513-38DA-E053-6C04A8C051AE},295000,2018-02-20 00:00,LE2 5PF,D,N,F,90,Unnamed: 8,HIDCOTE ROAD,OADBY,LEICESTER,OADBY AND WIGSTON,LEICESTERSHIRE,A,A.1
0,{68FEB20B-A514-38DA-E053-6C04A8C051AE},450000,2018-03-15 00:00,LE9 9FH,D,N,F,MARKFIELD LODGE,NaN,MARKFIELD LANE,BOTCHESTON,LEICESTER,HINCKLEY AND BOSWORTH,LEICESTERSHIRE,A,A
1,{68FEB20B-A515-38DA-E053-6C04A8C051AE},980000,2018-02-12 00:00,LE10 3AB,D,N,F,COTTAGE FARM,NaN,FOSSEWAY,SHARNFORD,HINCKLEY,BLABY,LEICESTERSHIRE,A,A
2,{68FEB20B-A516-38DA-E053-6C04A8C051AE},158000,2018-03-01 00:00,LE2 8FQ,T,N,F,69,NaN,PERCY ROAD,NaN,LEICESTER,LEICESTER,LEICESTER,A,A
3,{68FEB20B-A518-38DA-E053-6C04A8C051AE},240000,2018-02-12 00:00,LE11 2HN,D,N,F,360,NaN,PARK ROAD,NaN,LOUGHBOROUGH,CHARNWOOD,LEICESTERSHIRE,A,A
4,{68FEB20B-A519-38DA-E053-6C04A8C051AE},150000,2018-02-09 00:00,LE2 8QB,S,N,F,13,NaN,ALTON ROAD,NaN,LEICESTER,LEICESTER,LEICESTER,A,A


In [4]:
df_ppd.shape

(971140, 16)

Our dataset consists of over 971140 rows and 16 columns. We will now prepare and preprocess data accordingly.

# 3. Data preparation and preprocessing
At this stage, we prepare our dataset for the modeling process, opting for the most suitable machine learning algorithm for our scope. Accordingly, we perform the following steps:

Rename the column names
Format the date column
Sort data by date of sale
Select data only for the city of London
Make a list of street names in London
Calculate the street-wise average price of the property
Read the street-wise coordinates into a data frame, eliminating recurring word London from individual names
Join the data to find the coordinates of locations which fit into client's budget
Plot recommended locations on London map along with current market prices

In [5]:
# Assign meaningful column names
df_ppd.columns = ['TUID', 'Price', 'Date_Transfer', 'Postcode', 'Prop_Type', 'Old_New', 'Duration', 'PAON', \
                  'SAON', 'Street', 'Locality', 'Town_City', 'District', 'County', 'PPD_Cat_Type', 'Record_Status']

In [9]:
# Format the date column
df_ppd['Date_Transfer'] = df_ppd['Date_Transfer'].apply(pd.to_datetime)

# Delete all obsolete transactions which were done before 2016
df_ppd.drop(df_ppd[df_ppd.Date_Transfer.dt.year < 2016].index, inplace=True)

# Sort by Date of Sale
df_ppd.sort_values(by=['Date_Transfer'],ascending=[False],inplace=True)

In [10]:
df_ppd_london = df_ppd.query("Town_City == 'LONDON'")

# Make a list of street names in LONDON
streets = df_ppd_london['Street'].unique().tolist()

In [11]:
df_grp_price = df_ppd_london.groupby(['Street'])['Price'].mean().reset_index()

# Give meaningful names to the columns
df_grp_price.columns = ['Street', 'Avg_Price']

In [12]:
#Input your Budget's Upper Limit and Lower Limit - Find the locations df_grp_price which fits your budget
df_affordable = df_grp_price.query("(Avg_Price >= 2200000) & (Avg_Price <= 2500000)")

In [13]:
# Display the dataframe
df_affordable

,Street,Avg_Price
192,ALBION SQUARE,2.450000e+06
384,ANHALT ROAD,2.435000e+06
399,ANSDELL TERRACE,2.250000e+06
413,APPLEGARTH ROAD,2.400000e+06
685,AYLESTONE AVENUE,2.286667e+06
838,BARONSMEAD ROAD,2.375000e+06
961,BEAUCLERC ROAD,2.480000e+06
1082,BELVEDERE DRIVE,2.340000e+06
1195,BICKENHALL STREET,2.208500e+06
1233,BIRCHLANDS AVENUE,2.217000e+06


In [14]:
import pandas as pd
import numpy as np
import datetime as DT
import hmac
from geopy.geocoders import Nominatim
from geopy.distance import vincenty
# import k-means from clustering stage
from sklearn.cluster import KMeans

In [15]:
for index, item in df_affordable.iterrows():
    print(f"index: {index}")
    print(f"item: {item}")
    print(f"item.Street only: {item.Street}")

index: 192
item: Street       ALBION SQUARE
Avg_Price         2.45e+06
Name: 192, dtype: object
item.Street only: ALBION SQUARE
index: 384
item: Street       ANHALT ROAD
Avg_Price      2.435e+06
Name: 384, dtype: object
item.Street only: ANHALT ROAD
index: 399
item: Street       ANSDELL TERRACE
Avg_Price           2.25e+06
Name: 399, dtype: object
item.Street only: ANSDELL TERRACE
index: 413
item: Street       APPLEGARTH ROAD
Avg_Price            2.4e+06
Name: 413, dtype: object
item.Street only: APPLEGARTH ROAD
index: 685
item: Street       AYLESTONE AVENUE
Avg_Price         2.28667e+06
Name: 685, dtype: object
item.Street only: AYLESTONE AVENUE
index: 838
item: Street       BARONSMEAD ROAD
Avg_Price          2.375e+06
Name: 838, dtype: object
item.Street only: BARONSMEAD ROAD
index: 961
item: Street       BEAUCLERC ROAD
Avg_Price          2.48e+06
Name: 961, dtype: object
item.Street only: BEAUCLERC ROAD
index: 1082
item: Street       BELVEDERE DRIVE
Avg_Price           2.34e+06
Name

In [16]:
geolocator = Nominatim()

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  """Entry point for launching an IPython kernel.


In [17]:
df_affordable['city_coord'] = df_affordable['Street'].apply(geolocator.geocode).apply(lambda x: (x.latitude, x.longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [18]:
df_affordable

,Street,Avg_Price,city_coord
192,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)"
384,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)"
399,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)"
413,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)"
685,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)"
838,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)"
961,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)"
1082,BELVEDERE DRIVE,2.340000e+06,"(34.966857, -81.992424)"
1195,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)"
1233,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)"


In [19]:
df_affordable[['Latitude', 'Longitude']] = df_affordable['city_coord'].apply(pd.Series)

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3391: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[k1] = value[k2]


In [20]:
df_affordable

,Street,Avg_Price,city_coord,Latitude,Longitude
192,ALBION SQUARE,2.450000e+06,"(-41.27375755, 173.289393239104)",-41.273758,173.289393
384,ANHALT ROAD,2.435000e+06,"(51.4803265, -0.1667607)",51.480326,-0.166761
399,ANSDELL TERRACE,2.250000e+06,"(51.4998899, -0.1891027)",51.499890,-0.189103
413,APPLEGARTH ROAD,2.400000e+06,"(53.7486539, -0.3266704)",53.748654,-0.326670
685,AYLESTONE AVENUE,2.286667e+06,"(51.5409157, -0.2178742)",51.540916,-0.217874
838,BARONSMEAD ROAD,2.375000e+06,"(51.4773147, -0.239457)",51.477315,-0.239457
961,BEAUCLERC ROAD,2.480000e+06,"(51.4995771, -0.2290331)",51.499577,-0.229033
1082,BELVEDERE DRIVE,2.340000e+06,"(34.966857, -81.992424)",34.966857,-81.992424
1195,BICKENHALL STREET,2.208500e+06,"(51.5211969, -0.1589341)",51.521197,-0.158934
1233,BIRCHLANDS AVENUE,2.217000e+06,"(51.4483941, -0.1604676)",51.448394,-0.160468


In [21]:
df = df_affordable.drop(columns=['city_coord'])

In [22]:
df

,Street,Avg_Price,Latitude,Longitude
192,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
384,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
399,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
413,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
685,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874
838,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457
961,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033
1082,BELVEDERE DRIVE,2.340000e+06,34.966857,-81.992424
1195,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934
1233,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468


In [23]:
address = 'London, UK'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of London City are {}, {}.'.format(latitude, longitude))

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.18.1" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  This is separate from the ipykernel package so we can avoid doing imports until


The geograpical coordinate of London City are 51.5073219, -0.1276474.


In [24]:
# create map of London using latitude and longitude values
map_london = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, price, street in zip(df['Latitude'], df['Longitude'], df['Avg_Price'], df['Street']):
    label = '{}, {}'.format(street, price)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_london)  
    
map_london

In [25]:
#Define Foursquare Credentials and Version

CLIENT_ID = 'VP0505UDJZH40US0YKW5YPEQNY2GD5EXGGTZWJQPEHKRJGN2' # Foursquare ID
CLIENT_SECRET = 'D1ZABF3LVHG2GDFQ14SIRYZ24UQB1WDEVSHBR1EKFL55SAOB' # Foursquare Secret
VERSION = '20180604' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: VP0505UDJZH40US0YKW5YPEQNY2GD5EXGGTZWJQPEHKRJGN2
CLIENT_SECRET:D1ZABF3LVHG2GDFQ14SIRYZ24UQB1WDEVSHBR1EKFL55SAOB


We can now proceed to the Modeling phase. We will analyze neighborhoods to recommend real estates where home buyers can make a real estate investment. We will then recommend profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

# 4. Modeling
After exploring the dataset and gaining insights into it, we are ready to use the clustering methodology to analyze real estates. We will use the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

In [26]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, LIMIT=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Street', 
                  'Street Latitude', 
                  'Street Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [27]:
# Run the above function on each location and create a new dataframe called location_venues and display it.
location_venues = getNearbyVenues(names=df['Street'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

ALBION SQUARE
ANHALT ROAD
ANSDELL TERRACE
APPLEGARTH ROAD
AYLESTONE AVENUE
BARONSMEAD ROAD
BEAUCLERC ROAD
BELVEDERE DRIVE
BICKENHALL STREET
BIRCHLANDS AVENUE
BRAMPTON GROVE
BRIARDALE GARDENS
BROOKWAY
BURBAGE ROAD
BURY WALK
CALEDONIAN ROAD
CALLCOTT STREET
CAMPDEN HILL ROAD
CAMPION ROAD
CANNING PLACE
CARLISLE ROAD
CARLTON GARDENS
CARLYLE COURT
CHALCOT SQUARE
CHARLES LANE
CHELSEA CRESCENT
CHESTER CLOSE NORTH
CHEYNE COURT
CLARENDON STREET
CLONCURRY STREET
COLBECK MEWS
CORNWALL TERRACE MEWS
COURT LANE GARDENS
CRAVEN STREET
DALEBURY ROAD
DEWHURST ROAD
DORIA ROAD
DOWNSHIRE HILL
DUCHESS WALK
ECCLESTON SQUARE MEWS
EGBERT STREET
EGERTON PLACE
ELM PARK ROAD
FARM PLACE
FRANK DIXON WAY
FULTON MEWS
GERARD ROAD
GERRARD ROAD
GIBSON SQUARE
GIRDLERS ROAD
GLOUCESTER CRESCENT
GORDON PLACE
GRAFTON SQUARE
GRAHAM TERRACE
HARMAN DRIVE
HAVANNAH STREET
HAZLEWELL ROAD
HEREFORD MEWS
HERONDALE AVENUE
HIGHGATE HIGH STREET
HILLGATE PLACE
HOLLYCROFT AVENUE
HOLLYWOOD MEWS
HONEYWELL ROAD
HOXTON SQUARE
HUNTER ROAD
JACKS

In [28]:
location_venues

,Street,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ALBION SQUARE,-41.273758,173.289393,The Free House,-41.273340,173.287364,Bar
1,ALBION SQUARE,-41.273758,173.289393,The Indian Cafe,-41.273308,173.286530,Indian Restaurant
2,ALBION SQUARE,-41.273758,173.289393,Queen's Gardens,-41.273671,173.291383,Park
3,ALBION SQUARE,-41.273758,173.289393,Fish Stop,-41.276010,173.289592,Fish & Chips Shop
4,ALBION SQUARE,-41.273758,173.289393,Deville Cafe,-41.271941,173.285535,Beer Garden
5,ALBION SQUARE,-41.273758,173.289393,The Bridge Street Collective,-41.272520,173.285517,Café
6,ALBION SQUARE,-41.273758,173.289393,Urban,-41.274355,173.286317,New American Restaurant
7,ALBION SQUARE,-41.273758,173.289393,Burger Culture,-41.274750,173.284030,Burger Joint
8,ALBION SQUARE,-41.273758,173.289393,Fresh Choice,-41.272194,173.287218,Supermarket
9,ALBION SQUARE,-41.273758,173.289393,Mango,-41.274460,173.285345,Indian Restaurant


In [29]:
location_venues.groupby('Street').count()

,Street Latitude,Street Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Street,,,,,,
ALBION SQUARE,28,28,28,28,28,28
ANHALT ROAD,15,15,15,15,15,15
ANSDELL TERRACE,55,55,55,55,55,55
APPLEGARTH ROAD,4,4,4,4,4,4
AYLESTONE AVENUE,4,4,4,4,4,4
BARONSMEAD ROAD,15,15,15,15,15,15
BEAUCLERC ROAD,29,29,29,29,29,29
BELVEDERE DRIVE,2,2,2,2,2,2
BICKENHALL STREET,95,95,95,95,95,95


In [30]:
# get the List of Unique Categories
print('There are {} uniques categories.'.format(len(location_venues['Venue Category'].unique())))

There are 335 uniques categories.


In [31]:
# one hot encoding
venues_onehot = pd.get_dummies(location_venues[['Venue Category']], prefix="", prefix_sep="")

# add street column back to dataframe
venues_onehot['Street'] = location_venues['Street'] 

# move street column to the first column
fixed_columns = [venues_onehot.columns[-1]] + list(venues_onehot.columns[:-1])

#fixed_columns
venues_onehot = venues_onehot[fixed_columns]

venues_onehot.head()

,Street,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ALBION SQUARE,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
london_grouped = venues_onehot.groupby('Street').mean().reset_index()
london_grouped

,Street,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Waterfront,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,ALBION SQUARE,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.035714,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,ANHALT ROAD,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
2,ANSDELL TERRACE,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.018182,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.018182,0.000000
3,APPLEGARTH ROAD,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
4,AYLESTONE AVENUE,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
5,BARONSMEAD ROAD,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
6,BEAUCLERC ROAD,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,BELVEDERE DRIVE,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
8,BICKENHALL STREET,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.010526,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.010526,0.000000,0.000000,0.010526
9,BIRCHLANDS AVENUE,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,...,0.0,0.0000,0.0,0.0,0.000000,0.00,0.000000,0.000000,0.000000,0.000000


In [33]:
london_grouped.shape

(137, 336)

In [34]:
# What are the top 5 venues/facilities nearby profitable real estate investments?#

num_top_venues = 5

for hood in london_grouped['Street']:
    print("----"+hood+"----")
    temp = london_grouped[london_grouped['Street'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----ALBION SQUARE----
               venue  freq
0               Café  0.21
1  Indian Restaurant  0.11
2                Bar  0.07
3                Pub  0.07
4         Restaurant  0.07


----ANHALT ROAD----
           venue  freq
0            Pub  0.27
1  Grocery Store  0.13
2          Plaza  0.07
3         Garden  0.07
4   Cocktail Bar  0.07


----ANSDELL TERRACE----
                venue  freq
0  Italian Restaurant  0.07
1      Clothing Store  0.07
2                 Pub  0.05
3               Hotel  0.05
4          Restaurant  0.04


----APPLEGARTH ROAD----
               venue  freq
0          Nightclub  0.25
1                Pub  0.25
2                Bar  0.25
3             Casino  0.25
4  Accessories Store  0.00


----AYLESTONE AVENUE----
               venue  freq
0               Park  0.50
1           Bus Stop  0.25
2      Movie Theater  0.25
3  Accessories Store  0.00
4  Outdoor Sculpture  0.00


----BARONSMEAD ROAD----
              venue  freq
0        Restaurant  0.13
1      

In [35]:
# Define a function to return the most common venues/facilities nearby real estate investments#

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Street']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

In [37]:
# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['Street'] = london_grouped['Street']

for ind in np.arange(london_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(london_grouped.iloc[ind, :], num_top_venues)

In [38]:
venues_sorted.head()

,Street,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,ALBION SQUARE,Café,Indian Restaurant,Pub,Restaurant,Coffee Shop,Bar,Museum,Burger Joint,New American Restaurant,Brewery
1,ANHALT ROAD,Pub,Grocery Store,Garden,Chinese Restaurant,English Restaurant,Cocktail Bar,Diner,Pizza Place,Gym / Fitness Center,Japanese Restaurant
2,ANSDELL TERRACE,Italian Restaurant,Clothing Store,Hotel,Pub,Indian Restaurant,Bakery,Garden,Juice Bar,Café,Chinese Restaurant
3,APPLEGARTH ROAD,Pub,Bar,Nightclub,Casino,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
4,AYLESTONE AVENUE,Park,Movie Theater,Bus Stop,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop


In [39]:
venues_sorted.shape

(137, 11)

In [40]:
london_grouped.shape

(137, 336)

In [41]:
london_grouped=df

After our inspection of venues/facilities/amenities nearby the most profitable real estate investments in London, we could begin by clustering properties by venues/facilities/amenities nearby.

In [42]:
#Distribute in 5 Clusters

# set number of clusters
kclusters = 5

london_grouped_clustering = london_grouped.drop('Street', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(london_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:50]

array([4, 4, 3, 4, 1, 2, 0, 2, 3, 3, 0, 4, 4, 4, 0, 2, 2, 2, 0, 4, 3, 0,
       1, 1, 4, 0, 4, 3, 3, 2, 2, 2, 2, 2, 4, 4, 2, 3, 2, 2, 1, 3, 4, 1,
       3, 1, 3, 3, 3, 4], dtype=int32)

In [43]:
london_grouped_clustering=df
london_grouped_clustering.head()

,Street,Avg_Price,Latitude,Longitude
192,ALBION SQUARE,2.450000e+06,-41.273758,173.289393
384,ANHALT ROAD,2.435000e+06,51.480326,-0.166761
399,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103
413,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670
685,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874


In [44]:
london_grouped_clustering.shape

(148, 4)

In [45]:
df.shape

(148, 4)

In [46]:
london_grouped_clustering.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [47]:
df.dtypes

Street        object
Avg_Price    float64
Latitude     float64
Longitude    float64
dtype: object

In [48]:
# add clustering labels
london_grouped_clustering['Cluster Labels'] = kmeans.labels_

# merge london_grouped with london_data to add latitude/longitude for each neighborhood
london_grouped_clustering = london_grouped_clustering.join(venues_sorted.set_index('Street'), on='Street')

london_grouped_clustering.head(30) # check the last columns!

,Street,Avg_Price,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,ALBION SQUARE,2.450000e+06,-41.273758,173.289393,4,Café,Indian Restaurant,Pub,Restaurant,Coffee Shop,Bar,Museum,Burger Joint,New American Restaurant,Brewery
384,ANHALT ROAD,2.435000e+06,51.480326,-0.166761,4,Pub,Grocery Store,Garden,Chinese Restaurant,English Restaurant,Cocktail Bar,Diner,Pizza Place,Gym / Fitness Center,Japanese Restaurant
399,ANSDELL TERRACE,2.250000e+06,51.499890,-0.189103,3,Italian Restaurant,Clothing Store,Hotel,Pub,Indian Restaurant,Bakery,Garden,Juice Bar,Café,Chinese Restaurant
413,APPLEGARTH ROAD,2.400000e+06,53.748654,-0.326670,4,Pub,Bar,Nightclub,Casino,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
685,AYLESTONE AVENUE,2.286667e+06,51.540916,-0.217874,1,Park,Movie Theater,Bus Stop,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
838,BARONSMEAD ROAD,2.375000e+06,51.477315,-0.239457,2,Restaurant,Pub,Thai Restaurant,Pizza Place,Community Center,Coffee Shop,Park,Farmers Market,Food & Drink Shop,Café
961,BEAUCLERC ROAD,2.480000e+06,51.499577,-0.229033,0,Pub,Coffee Shop,Hotel,Thai Restaurant,Grocery Store,Gastropub,Chaat Place,Falafel Restaurant,Supermarket,Cocktail Bar
1082,BELVEDERE DRIVE,2.340000e+06,34.966857,-81.992424,2,Burger Joint,Discount Store,Friterie,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market
1195,BICKENHALL STREET,2.208500e+06,51.521197,-0.158934,3,Hotel,Café,Chinese Restaurant,Restaurant,Gastropub,Indian Restaurant,Coffee Shop,Italian Restaurant,Pub,Garden
1233,BIRCHLANDS AVENUE,2.217000e+06,51.448394,-0.160468,3,Pub,Chinese Restaurant,Train Station,French Restaurant,Bakery,Brewery,Breakfast Spot,Coffee Shop,Lake,Farm


In [49]:
# Create Map

map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(london_grouped_clustering['Latitude'], london_grouped_clustering['Longitude'], london_grouped_clustering['Street'], london_grouped_clustering['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [50]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 0, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
961,2480000.0,Pub,Coffee Shop,Hotel,Thai Restaurant,Grocery Store,Gastropub,Chaat Place,Falafel Restaurant,Supermarket,Cocktail Bar
1530,2456875.0,Middle Eastern Restaurant,Men's Store,Lake,Factory,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market
1949,2492500.0,Supermarket,English Restaurant,Pub,Park,American Restaurant,Café,Hardware Store,Rental Car Location,Gym,Yoga Studio
2103,2461000.0,Sports Club,Soccer Field,Windmill,Spa,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
2197,2483500.0,Italian Restaurant,Dessert Shop,Hotel,Pizza Place,Café,Convenience Store,Bar,Ice Cream Shop,Middle Eastern Restaurant,Coffee Shop


In [51]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 1, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
685,2.286667e+06,Park,Movie Theater,Bus Stop,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop
2209,2.300000e+06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2371,2.286679e+06,Café,Bar,Pizza Place,Italian Restaurant,Pub,Coffee Shop,Vegetarian / Vegan Restaurant,Restaurant,Cupcake Shop,Speakeasy
4221,2.265000e+06,Pub,Flea Market,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market
4631,2.300000e+06,Construction & Landscaping,Yoga Studio,Flower Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market


In [52]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 2, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
838,2.375000e+06,Restaurant,Pub,Thai Restaurant,Pizza Place,Community Center,Coffee Shop,Park,Farmers Market,Food & Drink Shop,Café
1082,2.340000e+06,Burger Joint,Discount Store,Friterie,Flower Shop,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market
2030,2.349182e+06,Grocery Store,Café,Park,Rental Car Location,Coffee Shop,Yoga Studio,Theater,Greek Restaurant,Farm,Ethiopian Restaurant
2035,2.375000e+06,Pub,Bakery,Yoga Studio,Indian Restaurant,Hotel,Italian Restaurant,Grocery Store,Park,Coffee Shop,Bookstore
2096,2.379653e+06,Pub,Bakery,Coffee Shop,Hotel,Grocery Store,Bookstore,Yoga Studio,Indian Restaurant,Hostel,Ice Cream Shop


In [53]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 3, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
399,2250000.0,Italian Restaurant,Clothing Store,Hotel,Pub,Indian Restaurant,Bakery,Garden,Juice Bar,Café,Chinese Restaurant
1195,2208500.0,Hotel,Café,Chinese Restaurant,Restaurant,Gastropub,Indian Restaurant,Coffee Shop,Italian Restaurant,Pub,Garden
1233,2217000.0,Pub,Chinese Restaurant,Train Station,French Restaurant,Bakery,Brewery,Breakfast Spot,Coffee Shop,Lake,Farm
2192,2200000.0,Playground,Nightlife Spot,Soccer Field,Yoga Studio,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
2600,2250000.0,Gastropub,Construction & Landscaping,Yoga Studio,Flea Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop


In [54]:
london_grouped_clustering.loc[london_grouped_clustering['Cluster Labels'] == 4, london_grouped_clustering.columns[[1] + list(range(5, london_grouped_clustering.shape[1]))]].head()

,Avg_Price,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
192,2450000.0,Café,Indian Restaurant,Pub,Restaurant,Coffee Shop,Bar,Museum,Burger Joint,New American Restaurant,Brewery
384,2435000.0,Pub,Grocery Store,Garden,Chinese Restaurant,English Restaurant,Cocktail Bar,Diner,Pizza Place,Gym / Fitness Center,Japanese Restaurant
413,2400000.0,Pub,Bar,Nightclub,Casino,Fish Market,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant
1609,2397132.0,Gym / Fitness Center,Health & Beauty Service,Grocery Store,Indian Restaurant,Coffee Shop,Yoga Studio,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market
1772,2400000.0,Park,Flea Market,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market


# Results and Discussion section

We may discuss our results under two main perspectives.

First, we may examine them according to neighborhoods/London areas. It is interesting to note that, although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair.

Second, we may analyze our results according to the five clusters we have produced. Even though, all clusters could praise an optimal range of facilities and amenities, we have found two main patterns. The first pattern we are referring to, i.e. Clusters 0, 2 and 4, may target home buyers prone to live in 'green' areas with parks, waterfronts. Instead, the second pattern we are referring to, i.e. Clusters 1 and 3, may target individuals who love pubs, theatres and soccer.

# Conclusion

To solve this above described business problem, we clustered London neighborhoods in order to recommend venues and the current average price of real estate where homebuyers can make a real estate investment. We recommended profitable venues according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores.

First, we gathered data on London properties and the relative price paid data were extracted from the HM Land Registry (http://landregistry.data.gov.uk/). Moreover, to explore and target recommended locations across different venues according to the presence of amenities and essential facilities, we accessed data through FourSquare API interface and arranged them as a data frame for visualization. By merging data on London properties and the relative price paid data from the HM Land Registry and data on amenities and essential facilities surrounding such properties from FourSquare API interface, we were able to recommend profitable real estate investments.

Second, The Methodology section comprised four stages: 1. Collect Inspection Data; 2. Explore and Understand Data; 3. Data preparation and preprocessing; 4. Modeling. In particular, in the modeling section, we used the k-means clustering technique as it is fast and efficient in terms of computational cost, is highly flexible to account for mutations in real estate market in London and is accurate.

Finally, we drew the conclusion that even though the London Housing Market may be in a rut, it is still an "ever-green" for business affairs. We discussed our results under two main perspectives. First, we examined them according to neighborhoods/London areas. although West London (Notting Hill, Kensington, Chelsea, Marylebone) and North-West London (Hampsted) might be considered highly profitable venues to purchase a real estate according to amenities and essential facilities surrounding such venues i.e. elementary schools, high schools, hospitals & grocery stores, South-West London (Wandsworth, Balham) and North-West London (Isliington) are arising as next future elite venues with a wide range of amenities and facilities. Accordingly, one might target under-priced real estates in these areas of London in order to make a business affair. Second, we analyzed our results according to the five clusters we produced. While Clusters 0, 2 and 4 may target home buyers prone to live in 'green' areas with parks, waterfronts, Clusters 1 and 3 may target individuals who love pubs, theatres and soccer.